In [397]:
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px 
import glob
import numpy as np

In [398]:
NODE_COUNT=2
USE_META_LEARNING=False

In [399]:
if USE_META_LEARNING:
    results = [r for r in glob.glob(f"./../results/*AVERAGE_AGGREGATION_{NODE_COUNT}*/") if not "CLIPPED" in r]
    plot_file_term="MetaL_method"
else:
    results = glob.glob(f"./../results/NO_META_LEARNING*{NODE_COUNT}*/")
    plot_file_term="FL_method"
len(results)

8

In [400]:
results

['./../results/NO_META_LEARNING_DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_2_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_RECENCY_2_NODES/',
 './../results/NO_META_LEARNING_MEDIAN_AGGREGATION_2_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_PERFORMANCE_SCORES_2_NODES/',
 './../results/NO_META_LEARNING_AVERAGE_AGGREGATION_2_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_PUNISHING_UPDATES_2_NODES/',
 './../results/NO_META_LEARNING_FIXED_CLIPPED_AVERAGE_AGGREGATION_2_NODES/',
 './../results/NO_META_LEARNING_NO_FEDERATED_LEARNING_2_NODES/']

In [401]:
iter_dfs = [pickle.load(open(file +"/iter_df.pkl","rb")) for file in results]
training_dfs = [pickle.load(open(file +"/training_df.pkl","rb")) for file in results]

if not USE_META_LEARNING:
    aggregation_methods=["_".join(file.split("_")[3:-2]) for file in results]
else:
    aggregation_methods=["_".join(file.split("/")[-2].split("_")[:-4]) for file in results]

In [402]:
aggregation_methods

['DYNAMIC_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_RECENCY',
 'MEDIAN_AGGREGATION',
 'WEIGHTED_AVERAGE_PERFORMANCE_SCORES',
 'AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PUNISHING_UPDATES',
 'FIXED_CLIPPED_AVERAGE_AGGREGATION',
 'NO_FEDERATED_LEARNING']

In [403]:
iter_dfs[0]

,curr_score_0,iter_counter_0,DG_rotation_0,timestamp_0,curr_score_1,iter_counter_1,DG_rotation_1,timestamp_1
0,NaN,1,0,2024-02-19 13:44:49.645412,NaN,1,1,2024-02-19 13:44:49.754145
1,0.784667,2,1,2024-02-19 13:44:57.975482,0.765333,2,2,2024-02-19 13:44:54.139613
2,0.789667,3,1,2024-02-19 13:44:58.593814,0.771667,3,3,2024-02-19 13:44:54.766253
3,0.785333,4,1,2024-02-19 13:44:59.213703,0.763667,4,4,2024-02-19 13:44:55.401041
4,0.788333,5,1,2024-02-19 13:44:59.837433,0.760000,5,4,2024-02-19 13:44:56.023065
...,...,...,...,...,...,...,...,...
995,0.775667,996,319,2024-02-19 14:01:11.352709,0.776333,996,449,2024-02-19 14:03:48.734164
996,0.757000,997,320,2024-02-19 14:01:12.156430,0.772667,997,450,2024-02-19 14:03:49.531723
997,0.741667,998,321,2024-02-19 14:01:12.957426,0.761333,998,451,2024-02-19 14:03:50.229243
998,0.746667,999,321,2024-02-19 14:01:13.743860,0.748333,999,452,2024-02-19 14:03:50.960101


In [404]:
scores=[]
for aggregation_method,iter_df in zip(aggregation_methods,iter_dfs):
    rel_columns= [c for c in iter_df.columns if "curr_score_" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = iter_df[rel_columns]
    scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [405]:
score_df=pd.concat(scores,axis=1)
score_df

,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,WEIGHTED_AVERAGE_RECENCY_curr_score_0,WEIGHTED_AVERAGE_RECENCY_curr_score_1,MEDIAN_AGGREGATION_curr_score_0,MEDIAN_AGGREGATION_curr_score_1,WEIGHTED_AVERAGE_PERFORMANCE_SCORES_curr_score_0,WEIGHTED_AVERAGE_PERFORMANCE_SCORES_curr_score_1,AVERAGE_AGGREGATION_curr_score_0,AVERAGE_AGGREGATION_curr_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_0,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_1,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,NO_FEDERATED_LEARNING_curr_score_0,NO_FEDERATED_LEARNING_curr_score_1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.784667,0.765333,0.787000,0.805667,0.776000,0.820000,0.809667,0.773333,0.796333,0.802000,0.798000,0.769333,0.809667,0.773333,0.773333,0.809667
2,0.789667,0.771667,0.784333,0.805333,0.779333,0.824000,0.806333,0.777333,0.798667,0.803333,0.805000,0.770333,0.806333,0.777333,0.777333,0.806333
3,0.785333,0.763667,0.782000,0.805333,0.779667,0.820000,0.798333,0.777667,0.798667,0.798333,0.800333,0.759000,0.798333,0.777667,0.777667,0.798333
4,0.788333,0.760000,0.770333,0.807000,0.776000,0.819667,0.803000,0.775333,0.802333,0.799333,0.806333,0.744333,0.803000,0.775333,0.775333,0.803000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.775667,0.776333,0.805667,0.724667,0.773333,0.790667,0.764667,0.829667,0.797667,0.817000,0.802333,0.783000,0.740333,0.758000,0.828333,0.895000
996,0.757000,0.772667,0.766000,0.723667,0.774333,0.792667,0.761333,0.828000,0.810000,0.802667,0.801000,0.778000,0.737667,0.757667,0.831667,0.889333
997,0.741667,0.761333,0.880333,0.722667,0.773667,0.788667,0.768000,0.829333,0.793333,0.790667,0.788667,0.776333,0.739333,0.761333,0.832333,0.889333
998,0.746667,0.748333,0.875667,0.696000,0.773000,0.781000,0.769333,0.826333,0.794333,0.790333,0.761000,0.774000,0.736667,0.757333,0.829333,0.890667


In [406]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    rel_col=[c for c in score_df.columns if aggregation_method == c[:-13]]
    result_dict=dict()
    by_node_result_average_dict=dict()
    for col in rel_col:
        by_node_result_average_dict[col.split("_")[-1]]=score_df[col].mean()
    result_dict["by_node_results"]=score_df[rel_col].mean(axis=0)
    result_dict["by_node_results_std"]=score_df[rel_col].std(axis=0)
    result_dict["total_average"]=score_df[rel_col].mean().mean()
    result_dict["total_std"]=score_df[rel_col].mean().std()
    result_dict["average_in_time"]= score_df[rel_col].mean(axis=1)
    average_score_dict[aggregation_method]=result_dict

In [407]:
aggregation_names=[]
aggregation_averages=[]
aggregation_stds=[]
by_node_averages=[]
by_node_stds=[]
average_in_time=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    aggregation_averages.append(val["total_average"])
    aggregation_stds.append(val["total_std"])
    by_node_averages.append(val["by_node_results"].values)
    by_node_stds.append(val["by_node_results_std"])
    average_in_time.append(val["average_in_time"])
sorted_tuples=sorted(zip(aggregation_names,aggregation_averages,by_node_averages,average_in_time,aggregation_stds, by_node_stds),key=lambda x: x[1])
aggregation_names = [val[0] for val in sorted_tuples]
aggregation_averages = [val[1] for val in sorted_tuples]
by_node_averages = [val[2] for val in sorted_tuples]
average_in_time = [val[3] for val in sorted_tuples]
aggregation_stds = [val[4] for val in sorted_tuples]
by_node_stds = [val[5] for val in sorted_tuples]

In [408]:
n_nodes=len(by_node_averages[0])
n_nodes

2

In [409]:
by_node_averages

[array([0.7298345, 0.723003 ]),
 array([0.7483627, 0.7414381]),
 array([0.77907741, 0.74770838]),
 array([0.75659092, 0.77398265]),
 array([0.76812479, 0.78218185]),
 array([0.79568735, 0.762002  ]),
 array([0.80384618, 0.77941175]),
 array([0.7691368 , 0.81467568])]

In [410]:
processed_aggregation_names=[" ".join(name.split("_")[:-1]) for name in aggregation_names]
processed_aggregation_names

['MEDIAN',
 'DYNAMIC CLIPPED AVERAGE',
 'WEIGHTED AVERAGE PUNISHING',
 'FIXED CLIPPED AVERAGE',
 'WEIGHTED AVERAGE PERFORMANCE',
 'AVERAGE',
 'WEIGHTED AVERAGE',
 'NO FEDERATED']

In [411]:
aggregation_names

['MEDIAN_AGGREGATION',
 'DYNAMIC_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PUNISHING_UPDATES',
 'FIXED_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PERFORMANCE_SCORES',
 'AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_RECENCY',
 'NO_FEDERATED_LEARNING']

In [412]:
name_mapping={"FIXED_CLIPPED_AVERAGE_AGGREGATION": "Fixed clip",
 "MEDIAN_AGGREGATION": "Median",
 "DYNAMIC_CLIPPED_AVERAGE_AGGREGATION": "Dynamic clip",
 "WEIGHTED_AVERAGE_PUNISHING_UPDATES": "Punish updates",
 "AVERAGE_AGGREGATION": "Simple average",
 "WEIGHTED_AVERAGE_RECENCY": "Receny weighted",
 "WEIGHTED_AVERAGE_PERFORMANCE_SCORES": "Performance weighted",
 "NO_FEDERATED_LEARNING": "No FL",
 "NO_META_LEARNING": "No MetaL",
 "DIRECT_SCORE_WEIGHTED_MEANS":"Direct weighted",
 "DIRECT_GENETIC_ALGORITHM_MATING":"Direct sampled",
 "GRADUAL_SCORE_WEIGHTED_MEANS":"Gradual weighted",
 "GRADUAL_MEANS":"Gradual regular",
 "GRADUAL_GENETIC_ALGORITHM_MATING":"Gradual sampled",
 "DIRECT_MEANS":"Direct regular",}
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Median',
 'Dynamic clip',
 'Punish updates',
 'Fixed clip',
 'Performance weighted',
 'Simple average',
 'Receny weighted',
 'No FL']

In [413]:
fig=go.Figure()
for node in range(n_nodes):
    fig.add_trace(go.Bar(x=processed_aggregation_names, y=[val[node] for val in by_node_averages],error_y=dict(type="data",array=[val[node] for val in by_node_stds]),name=f"Node {node}"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=aggregation_averages,error_y=dict(type="data",array=aggregation_stds),name="Average"))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Average_score_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

/tmp/ipykernel_1766271/1946294216.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [414]:
fig=go.Figure()
for idx,name in enumerate(processed_aggregation_names):
    fig.add_trace(go.Scatter(y=average_in_time[idx][1:] ,name=name))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=600,legend=dict( orientation="h",y=1.2),margin=dict(t=150,r=10))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Performance_in_time_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [415]:
training_dfs[0].head()

,batch_size_mean_0,batch_size_std_0,learning_rate_mean_0,learning_rate_std_0,nesterov_mean_0,nesterov_std_0,momentum_mean_0,momentum_std_0,n_runs_mean_0,n_runs_std_0,...,n_runs_1,n_epochs_1,weights_1,iter_count_1,pre_training_score_1,scores_1,best_score_1,best_hyper_params_1,best_model_weights_1,timestamp_1
0,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,4,10,"[[[-0.1033040402457118, -0.12606027899309993, ...",505,0.428333,"[0.7683333333333333, 0.7583333333333333, 0.775...",0.775000,"{'learning_rate': 0.0005776896579256762, 'batc...","[[[-0.2900177836418152, -0.35208454728126526, ...",2024-02-19 13:53:16.165681
1,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,10,8,"[[[-0.44128153957426547, -0.863507616519928, 1...",978,0.561667,"[0.77, 0.7716666666666666, 0.7666666666666667,...",0.771667,"{'learning_rate': 2.626566742616647e-05, 'batc...","[[[-0.4006930887699127, -0.8658930063247681, 2...",2024-02-19 14:03:22.364555
2,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,10,9,"[[[0.1397607061598036, 0.0736977621499035, 0.7...",102,0.531667,"[0.8016666666666666, 0.7816666666666666, 0.803...",0.806667,"{'learning_rate': 0.007297375757942594, 'batch...","[[[0.08281261473894119, 0.008993975818157196, ...",2024-02-19 13:46:29.629368
3,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,10,6,"[[[-0.4206873714923859, -0.5768155358731747, 1...",832,0.661667,"[0.7583333333333333, 0.755, 0.7566666666666667...",0.761667,"{'learning_rate': 0.00010619047591506953, 'bat...","[[[-0.4436478316783905, -1.0493214130401611, 1...",2024-02-19 13:59:54.114885
4,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,4,10,None,1,0.503333,"[0.755, 0.745, 0.6066666666666667, 0.591666666...",0.755000,"{'learning_rate': 1.0375363398017854e-05, 'bat...","[[[0.6922287344932556, 0.193614199757576, -0.2...",2024-02-19 13:44:53.496858


In [416]:
trainig_scores=[]
for aggregation_method,training_df in zip(aggregation_methods,training_dfs):
    rel_columns= [c for c in training_df.columns if "score" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = training_df[rel_columns]
    trainig_scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [417]:
training_scores_df=pd.concat(trainig_scores,axis=1)
training_scores_df

,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_pre_training_score_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_scores_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_best_score_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_pre_training_score_1,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_scores_1,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_best_score_1,WEIGHTED_AVERAGE_RECENCY_pre_training_score_0,WEIGHTED_AVERAGE_RECENCY_scores_0,WEIGHTED_AVERAGE_RECENCY_best_score_0,WEIGHTED_AVERAGE_RECENCY_pre_training_score_1,...,FIXED_CLIPPED_AVERAGE_AGGREGATION_best_score_0,FIXED_CLIPPED_AVERAGE_AGGREGATION_pre_training_score_1,FIXED_CLIPPED_AVERAGE_AGGREGATION_scores_1,FIXED_CLIPPED_AVERAGE_AGGREGATION_best_score_1,NO_FEDERATED_LEARNING_pre_training_score_0,NO_FEDERATED_LEARNING_scores_0,NO_FEDERATED_LEARNING_best_score_0,NO_FEDERATED_LEARNING_pre_training_score_1,NO_FEDERATED_LEARNING_scores_1,NO_FEDERATED_LEARNING_best_score_1
0,0.655000,"[0.8083333333333333, 0.79, 0.8033333333333333,...",0.808333,0.428333,"[0.7683333333333333, 0.7583333333333333, 0.775...",0.775000,0.693333,"[0.8, 0.7983333333333333, 0.7983333333333333, ...",0.803333,0.515000,...,0.816667,0.515000,"[0.7366666666666667, 0.7566666666666667, 0.6, ...",0.756667,0.743333,"[0.8333333333333334, 0.815, 0.8133333333333334...",0.833333,0.763333,"[0.8366666666666667, 0.8116666666666666, 0.786...",0.843333
1,0.506667,"[0.7533333333333333, 0.76, 0.77, 0.75, 0.75833...",0.770000,0.561667,"[0.77, 0.7716666666666666, 0.7666666666666667,...",0.771667,0.758333,"[0.8033333333333333, 0.8466666666666667, 0.811...",0.850000,0.541667,...,0.821667,0.445000,"[0.76, 0.7583333333333333, 0.7816666666666666,...",0.781667,0.733333,"[0.8, 0.8133333333333334, 0.7766666666666666, ...",0.813333,0.806667,"[0.8883333333333333, 0.8866666666666667, 0.888...",0.898333
2,0.756667,"[0.8216666666666667, 0.8016666666666666, 0.82,...",0.821667,0.531667,"[0.8016666666666666, 0.7816666666666666, 0.803...",0.806667,0.783333,"[0.8783333333333333, 0.8633333333333333, 0.888...",0.890000,0.486667,...,0.763333,0.475000,"[0.775, 0.77, 0.7716666666666666, 0.78, 0.7783...",0.780000,0.710000,"[0.7933333333333333, 0.7983333333333333, 0.793...",0.798333,0.755000,"[0.845, 0.8383333333333334, 0.8566666666666667...",0.856667
3,0.681667,"[0.78, 0.7883333333333333, 0.7766666666666666,...",0.788333,0.661667,"[0.7583333333333333, 0.755, 0.7566666666666667...",0.761667,0.776667,"[0.8066666666666666, 0.7816666666666666, 0.756...",0.806667,0.801667,...,0.806667,0.400000,"[0.7816666666666666, 0.7616666666666667, 0.778...",0.801667,0.676667,"[0.7933333333333333, 0.7683333333333333, 0.763...",0.811667,0.781667,"[0.885, 0.885, 0.8866666666666667, 0.89]",0.890000
4,0.661667,"[0.7366666666666667, 0.7316666666666667, 0.731...",0.748333,0.503333,"[0.755, 0.745, 0.6066666666666667, 0.591666666...",0.755000,0.701667,"[0.785, 0.7733333333333333, 0.7716666666666666...",0.810000,0.530000,...,0.806667,0.488333,"[0.7616666666666667, 0.7633333333333333, 0.76,...",0.768333,0.705000,"[0.7566666666666667, 0.7616666666666667, 0.768...",0.768333,0.791667,"[0.8016666666666666, 0.8066666666666666, 0.87,...",0.870000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,0.778333,"[0.81, 0.8233333333333334, 0.8166666666666667,...",0.830000
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,0.796667,"[0.8716666666666667, 0.8516666666666667, 0.868...",0.871667
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,0.723333,"[0.8233333333333334, 0.7933333333333333, 0.808...",0.833333
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,0.795000,"[0.86, 0.8616666666666667, 0.8116666666666666,...",0.868333


In [418]:
len("_scores_0")

9

In [419]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    print(aggregation_method)
    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-21]]
    sub_filtered_rel_cols=[c for c in rel_col if "pre_training" in c]
    result_dict=dict()
    result_dict["by_node_results_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-13]]
    sub_filtered_rel_cols=[c for c in rel_col if "best_score" in c]
    result_dict["by_node_results_best_score"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_best_score"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-9]]
    sub_filtered_rel_cols=[c for c in rel_col if "scores" in c]
    by_node_result_average_dict=dict()
    by_node_result_std_dict=dict()
    by_node_retraining_count_dict=dict()
    by_node_training_count_dict=dict()
    for col in sub_filtered_rel_cols:
        by_node_result_average_dict[col.split("_")[-1]]=np.mean(np.concatenate(training_scores_df[col].dropna().values))
        by_node_result_std_dict[col.split("_")[-1]]=np.std(np.concatenate(training_scores_df[col].dropna().values))
        by_node_training_count_dict[col.split("_")[-1]]=len(np.concatenate(training_scores_df[col].dropna().values))
        by_node_retraining_count_dict[col.split("_")[-1]]= len(training_scores_df[col].dropna().values)
    result_dict["by_node_results_average_scores"]=by_node_result_average_dict
    result_dict["total_average_average_scores"]=np.mean([val for val in by_node_result_average_dict.values()])
    result_dict["by_node_results_std_scores"]=by_node_result_std_dict
    result_dict["total_average_std_scores"]=np.mean([val for val in by_node_result_std_dict.values()])
    result_dict["by_node_results_retraining_count"]=by_node_retraining_count_dict
    result_dict["total_retraining_count"]=np.sum([val for val in by_node_retraining_count_dict.values()])
    result_dict["by_node_results_training_count"]=by_node_training_count_dict
    result_dict["total_training_count"]=np.sum([val for val in by_node_training_count_dict.values()])
    average_score_dict[aggregation_method]=result_dict

DYNAMIC_CLIPPED_AVERAGE_AGGREGATION
WEIGHTED_AVERAGE_RECENCY
MEDIAN_AGGREGATION
WEIGHTED_AVERAGE_PERFORMANCE_SCORES
AVERAGE_AGGREGATION
WEIGHTED_AVERAGE_PUNISHING_UPDATES
FIXED_CLIPPED_AVERAGE_AGGREGATION
NO_FEDERATED_LEARNING


In [420]:
total_average_pre_training = []
total_average_best_score = []
total_average_std_scores = []
total_retraining_count = []
total_training_count = []
total_average_average_scores = []
aggregation_names=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    total_average_pre_training.append(val["total_average_pre_training"])
    total_average_best_score.append(val["total_average_best_score"])
    total_average_std_scores.append(val["total_average_std_scores"])
    total_retraining_count.append(val["total_retraining_count"])
    total_training_count.append(val["total_training_count"])
    total_average_average_scores.append(val["total_average_average_scores"])
print(list(zip(aggregation_names,total_training_count)))
sorted_tuples = sorted(
    zip(
        aggregation_names,
        total_average_pre_training,
        total_average_best_score,
        total_average_std_scores,
        total_retraining_count,
        total_training_count,
        total_average_average_scores,
    ),
    key=lambda x: x[2],
)
aggregation_names = [val[0] for val in sorted_tuples]
total_average_pre_training = [val[1] for val in sorted_tuples]
total_average_best_score = [val[2] for val in sorted_tuples]
total_average_std_scores = [val[3] for val in sorted_tuples]
total_retraining_count = [val[4] for val in sorted_tuples]
total_training_count = [val[5] for val in sorted_tuples]
total_average_average_scores = [val[6] for val in sorted_tuples]

[('DYNAMIC_CLIPPED_AVERAGE_AGGREGATION', 608), ('WEIGHTED_AVERAGE_RECENCY', 1028), ('MEDIAN_AGGREGATION', 441), ('WEIGHTED_AVERAGE_PERFORMANCE_SCORES', 842), ('AVERAGE_AGGREGATION', 867), ('WEIGHTED_AVERAGE_PUNISHING_UPDATES', 1001), ('FIXED_CLIPPED_AVERAGE_AGGREGATION', 778), ('NO_FEDERATED_LEARNING', 1098)]


In [421]:
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Dynamic clip',
 'Median',
 'Fixed clip',
 'Punish updates',
 'Performance weighted',
 'Simple average',
 'Receny weighted',
 'No FL']

In [422]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_pre_training,name="Pre-training"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_best_score,name="Best"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_average_scores,name="Average"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_std_scores,name="Training std"))
fig.update_layout(template="none", title=f"Training results by aggregation methods for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Training_results_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [423]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_training_count,name="Trainings"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_retraining_count,name="Re-trainings"))
fig.update_layout(template="none", title=f"Training cost by aggregation methods for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Number of executions")
fig.write_image(f"./../figs/Training_cost_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()